In [1]:
%%HTML
<link rel="stylesheet" type="text/css" href="//fonts.googleapis.com/css?family=Quicksand:300" />
<link rel="stylesheet" type="text/css" href="custom.css">

# Context managers, the how and the why

In the decorators section, we saw some code such as this:
```python
try:
    cursor = connection.cursor()
    result = function(*args, **kwargs, cursor=cursor)
    connection.commit()
finally:
    cursor.close()
```        

The `finally` block ensures the `cursor.close()` code **always** run (it might throw some exceptions though :).

There are various scenario's where it's important to execute cleanup code *no matter* what happened. 

This is the reason Python introduced the `with` statement/context managers.

With context managers you can write your code as this

```python
with connection.cursor() as cursor:
    result = function(*args, **kwargs, cursor=cursor)
    connection.commit()
```

But the magic ends here: someone, somewhere, needs to tell the object (or function) that a cursor should be closed.

The `.cursor()` method of `connection` should therefore implement the context manager interface. In practice it works like this

```python
class Connection:
    ...
    def cursor(self, *args, **kwargs):
        return Cursor(*args, **kwargs)
    ...

class Cursor:
    ...
    def __enter__(self):
        return self
    
    def __exit__(self, *args):
        cursor.close()
    ...
```

## Exercise

Implement the `Printer` class to get the behavior as below

```python
>>> with Printer(text='hey') as p:
...    p.print_('you')
...    raise Exception
hey
you
goodbye
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
<ipython-input-6-26ab3e0380cc> in <module>()
      1 with Printer(text='hey') as p:
      2     p.print_('you')
----> 3     raise Exception

Exception:
```

If we want something more lightweight, there's also a context manager decorator

In [2]:
from contextlib import contextmanager

class Printer:
    def __init__(self, text=None):
        self.print_(text)

    def print_(self, text=None):
        if text:
            print(text)


@contextmanager
def printer(text):
    p = Printer(text)
    yield p
    p.print_('goodbye')
    
with printer(text='hey') as p:
    p.print_('you')

hey
you
goodbye


What's `yield`ed is what will be assigned after the `as`.

You can also `yield` nothing if you want

In [5]:
@contextmanager
def printer(p, text):
    p.print_(text)
    yield
    p.print_('goodbye')
    
p = Printer()
with printer(p, text='hey'):
    p.print_('you')

hey
you
goodbye


Note that exceptions could still happen in this case and you will not be protected from those. You need to explicitly guard you against those, so the cleanup code should be in a `finally`!

```python
@contextmanager
def printer(p, text):
    p.print_(text)
    try:
        yield
    finally:  # no except: the exception will still be raised, but the cleanup code will always execute!
        p.print_('goodbye')
```

## Exercise

change the `p.print_('goodbye')` inside `def printer(p, text)` to `p.print('goodbye')` and see what happens.

Putting context managers and decorators together is really powerful

```python
@contextmanager
def get_cursor(pool: AbstractConnectionPool, factory: Cursor=None):
    try:
        with pool.getconn() as conn, conn.cursor(cursor_factory=factory) as cur:
            yield cur
    finally:
        pool.putconn(conn)

```


```python
def manage_pool(factory: Cursor=None) -> Callable:
    def function(function):
        @wraps(function)
        def wrapper(*args, **kwargs):
            pool = kwargs.get('pool')
            with get_cursor(pool, factory) as cursor:
                result = function(*args, cursor=cursor, **kwargs)
            return result
        return wrapper
    return function
```

The machinery above might seem complicated, but *usage* is very simple:

```python
@manage_pool(factory=RealDictCursor)
def give_me_records(*, pool: AbstractConnectionPool, cursor: Cursor):
    cursor.execute("SELECT * FROM my_table")
    return cursor.fetchall()


# I need to pass a valid pool but I shouldn't pass a cursor!  
results = give_me_records(pool)
```

## Exercise

Why am I not passing `pool` to the decorator instead of passing it to the function?

Context managers are a bit overall in Python. Every time you open a file you should use a context manager:

Good:

```python
with open('my_file', 'r') as f:
    content = f.read()
```

Ugly:

```python
try:
    f = open('my_file', 'r')
    content = f.read()
finally:
    f.close()
```

Bad:

```python
f = open('my_file', 'r')
content = f.read()
f.close()
```

## Exercise

Write a context manager using `@contextmanager` to open files safely